In [1]:
from openagentkit.modules.openai import OpenAIExecutor
from openagentkit.core.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
import openai
import os

ModuleNotFoundError: No module named 'openagentkit'

In [ ]:
# Define a tool
@tool(
    description="Get the weather of a city", # Define the tool description
)
def get_weather(city: Annotated[str, "The city to get the weather of"]): # Each argument must be of type Annotated
    """Get the weather of a city"""

    # Actual implementation here...
    # ...

    return f"Weather in {city}: sunny, 20°C, feels like 22°C, humidity: 50%"

In [ ]:
get_weather.schema

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get the weather of a city',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to get the weather of'}},
   'required': ['city'],
   'additionalProperties': False}}}

In [ ]:
@tool(
    description="Calculate the sum of 2 numbers", # Define the tool description
)
def calculate(a: Annotated[int, "a"], b: Annotated[int, "b"]) -> int:
    """Calculate the sum of two numbers"""
    return a + b

In [ ]:
calculate.schema

{'type': 'function',
 'function': {'name': 'calculate',
  'description': 'Calculate the sum of 2 numbers',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': 'a'},
    'b': {'type': 'integer', 'description': 'b'}},
   'required': ['a', 'b'],
   'additionalProperties': False}}}

In [ ]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response)

2025-04-19 16:45:27.761 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_es5AuJzVHSB7PwFugHHLKjMI', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
2025-04-19 16:45:27.765 | INFO     | openagentkit.core.handlers.tool_handler:handle_tool_request:179 - Tool Result: Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%


role='assistant' content=None tool_calls=[{'id': 'call_es5AuJzVHSB7PwFugHHLKjMI', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
role='tool' content=None tool_calls=None tool_results=[ToolCallResult(tool_name='get_weather', result='Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%')] refusal=None audio=None usage=None


2025-04-19 16:45:28.743 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=162, completion_tokens=34, total_tokens=196, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


role='assistant' content='The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.' tool_calls=None tool_results=[] refusal=None audio=None usage=UsageResponse(prompt_tokens=162, completion_tokens=34, total_tokens=196, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


In [ ]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-19 16:45:26\n\n        \n        '},
 {'role': 'user', 'content': "What's the weather like in New York?"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_es5AuJzVHSB7PwFugHHLKjMI',
    'type': 'function',
    'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}],
  'content': 'None'},
 ToolCallMessage(role='tool', tool_call_id='call_es5AuJzVHSB7PwFugHHLKjMI', content='Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%'),
 {'role': 'assistant',
  'content': 'The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.'},
 {'role': 'assistant',
  'content': 'The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.'}]

In [ ]:
executor.clear_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-18 00:26:52\n\n        \n        '}]

In [ ]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    
    if user_input.lower() == "exit":
        break

    generator = executor.execute(
        messages=[
            {"role": "user", "content": user_input}
        ]
    )

    for response in generator:
        if response.content is not None:
            print(f"AI: {response.content}")
        if response.tool_calls:
            [print(f"Tool Calls: {tool_call}") for tool_call in response.tool_calls]

User: Hello


2025-04-18 00:26:59.921 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='Hello! How can I assist you today?' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=105, completion_tokens=11, total_tokens=116, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: Hello! How can I assist you today?
User: What's the weather like?


2025-04-18 00:27:05.276 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='Could you please specify which city you would like to know the weather for?' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=140, completion_tokens=17, total_tokens=157, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: Could you please specify which city you would like to know the weather for?
User: I'm in Hanoi at the moment


2025-04-18 00:27:14.227 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_bvMpEvk9pzX3hkYegJPU4qiF', 'type': 'function', 'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=188, completion_tokens=16, total_tokens=204, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
2025-04-18 00:27:14.228 | INFO     | openagentkit.core.handlers.tool_handler:handle_tool_request:179 - Tool Result: Weather in Hanoi: sunny, 20°C, feels like 22°C, humidity: 50%


Tool Calls: {'id': 'call_bvMpEvk9pzX3hkYegJPU4qiF', 'type': 'function', 'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}


2025-04-18 00:27:16.348 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='The weather in Hanoi is currently sunny, with a temperature of 20°C. It feels like 22°C, and the humidity is at 50%. If you need more information or have other questions, feel free to ask!' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=237, completion_tokens=48, total_tokens=285, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: The weather in Hanoi is currently sunny, with a temperature of 20°C. It feels like 22°C, and the humidity is at 50%. If you need more information or have other questions, feel free to ask!
User: exit


In [ ]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-18 00:26:52\n\n        \n        '},
 {'role': 'user', 'content': 'Hello'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user', 'content': "What's the weather like?"},
 {'role': 'assistant',
  'content': 'Could you please specify which city you would like to know the weather for?'},
 {'role': 'assistant',
  'content': 'Could you please specify which city you would like to know the weather for?'},
 {'role': 'user', 'content': "I'm in Hanoi at the moment"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_bvMpEvk9pzX3hkYegJPU4qiF',
    'type': 'function',
    'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}],
  'content': 'None'},
 ToolCall